In [1]:
!pip install pandas sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.0 MB/s eta 0:00:00


In [3]:
!pip install -q sentence-transformers faiss-cpu pandas langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss


In [7]:
# Mount Google Drive (or upload filtered_complaints.csv manually)
from google.colab import drive

# Load your filtered data
df = pd.read_csv('/content/filtered_complaints.csv')
texts = df['clean_narrative'].tolist()  # Use your cleaned column

In [8]:
# Initialize model ONCE (GPU-optimized)
embedder = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

# Batch processing for speed
batch_size = 256  # Max stable batch for Colab T4
embeddings = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    batch_embeds = embedder.encode(batch, show_progress_bar=True)
    embeddings.append(batch_embeds)

embeddings = np.vstack(embeddings)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

vector store


In [9]:
# Create index (faster than Chroma for Colab)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner product for cosine similarity
index.add(embeddings)

# Save index for Task 3
faiss.write_index(index, 'faiss_index.bin')

# Optional: Save embeddings for debugging
np.save('embeddings.npy', embeddings)

In [ ]:
verify retrieval works

In [10]:
# Test search
query = "unauthorized credit card charge"
query_embed = embedder.encode([query])
k = 3  # Number of results
D, I = index.search(query_embed, k)  # D = distances, I = indices

print("Top matches:")
for idx in I[0]:
    print(f"- {texts[idx][:100]}... (Product: {df.iloc[idx]['Product']})")

Top matches:
- my debit card was locked and a charge appeared on my locked card for xxxx xxxx for 900 i had a free ... (Product: Checking or savings account)
- xxxx credit card has charged me exponentially more than what my original borrow request was for afte... (Product: Credit card)
- i recieved a pre approved card in the mail i activated the card the card was loststolen the card was... (Product: Credit card)
